In [1]:
!pip install jieba

In [2]:
!pip install gensim

In [3]:
!pip install plotly

In [4]:
!pip install requests

In [5]:
!pip install feedparser

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 KB 1.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6066 sha256=d0af708489d8ea39e9686bf2081936b7517e9fe7cd50cb1738cb60a825bdc1a3
  Stored in directory: /home/jovyan/.cache/pip/wheels/65/7a/a7/78c287f64e401255dff4c13fdbc672fed5efbfd21c530114e1
Successfully built sgmllib3k


In [6]:
!pip install lxml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 13.8 MB/s eta 0:00:0000:0100:01


### 準備資料

In [7]:
import feedparser

In [8]:
rss_url = "https://money.udn.com/rssfeed/news/1001/5588/10511?ch=money"
rss_url

'https://money.udn.com/rssfeed/news/1001/5588/10511?ch=money'

In [9]:
import ssl
if hasattr(ssl, '_create_unverified_context'):
    ssl._create_default_https_context = ssl._create_unverified_context

In [10]:
newsFeed = feedparser.parse(rss_url)
newsFeed

{'bozo': False,
 'entries': [{'title': '美國防部：台灣主權若遭侵犯 烏克蘭是自衛良好典範',
   'title_detail': {'type': 'text/plain',
    'language': None,
    'base': 'https://money.udn.com/rssfeed/news/1001/5588/10511?ch=money',
    'value': '美國防部：台灣主權若遭侵犯 烏克蘭是自衛良好典範'},
   'links': [{'rel': 'alternate',
     'type': 'text/html',
     'href': 'https://money.udn.com/money/story/10511/6156480'}],
   'link': 'https://money.udn.com/money/story/10511/6156480',
   'published': 'Fri, 11 Mar 2022 07:09:23 +0800',
   'published_parsed': time.struct_time(tm_year=2022, tm_mon=3, tm_mday=10, tm_hour=23, tm_min=9, tm_sec=23, tm_wday=3, tm_yday=69, tm_isdst=0),
   'summary': '美國國防部助理部長卡琳（Mara Karlin） 周四在參議院聽證會上表示，若中國大陸選擇侵略侵犯台灣主權，烏克蘭對俄羅...',
   'summary_detail': {'type': 'text/html',
    'language': None,
    'base': 'https://money.udn.com/rssfeed/news/1001/5588/10511?ch=money',
    'value': '美國國防部助理部長卡琳（Mara Karlin） 周四在參議院聽證會上表示，若中國大陸選擇侵略侵犯台灣主權，烏克蘭對俄羅...'},
   'id': 'https://money.udn.com/money/story/10511/6156480',
   'gu

In [11]:
i = 1
for e in newsFeed['entries']:
    title = e['title']
    link_url = e['links'][0]['href']
    print("%s, %s, %s"%(i, title, link_url))
    i = i + 1

1, 美國防部：台灣主權若遭侵犯 烏克蘭是自衛良好典範, https://money.udn.com/money/story/10511/6156480
2, 匈牙利國會選出史上首位女總統 5月10日上任, https://money.udn.com/money/story/10511/6155939
3, 美眾議院通過預算案 金援烏克蘭約4000億元, https://money.udn.com/money/story/10511/6155038
4, 戰火毀烏克蘭藝術品 「和平鴿展翅」成挺烏象徵, https://money.udn.com/money/story/10511/6154694
5, 美眾院通過支出法案 禁「不正確標示」台灣領土, https://money.udn.com/money/story/10511/6154407
6, 川普有驚無險！專機半路故障緊急降落 借飛機返家, https://money.udn.com/money/story/10511/6154246
7, 發動機故障 川普搭乘私人飛機緊急迫降, https://money.udn.com/money/story/10511/6154166
8, 美佛州通過「不說同性戀」法  高中生罷課抗議, https://money.udn.com/money/story/10511/6153703
9, 準備生化戰？美警告：俄羅斯可能使用化學武器, https://money.udn.com/money/story/10511/6153696
10, 鄧振中：台灣想成為美印太經濟框架的「正式成員」, https://money.udn.com/money/story/10511/6153562
11, 中國美商會：美中緊張及防疫規範 令在華美企悲觀, https://money.udn.com/money/story/10511/6150147
12, 出於監管擔憂 在華美國企業不願擴大投資, https://money.udn.com/money/story/10511/6148009
13, 美中貿易戰 拜登無意停火, https://money.udn.com/money/story/10511/6145025
14, 美對陸懲罰關稅法定檢討時間將至  拜登政府無意解除, htt

In [12]:
import requests
from lxml import etree
from io import StringIO
import jieba
import pandas as pd
import time
import random
import pprint

each_article_text_list = []
news_crawl_result_file = 'news_crawl_result.txt'

In [13]:
#設定 request header
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36"
my_headers = {
    'User-Agent': user_agent,
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7",
    "accept-encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "cache-control": "no-cache",
    "Accept-Charset": "UTF8,utf-8;q=0.7,*;q=0.7"
}

In [14]:
#爬文
for e in newsFeed['entries']:
    url = e['links'][0]['href']
    print(url)
    r = requests.get(url, headers = my_headers,verify=False)
    if r.status_code == 200:
        parse_tree = etree.parse(StringIO(r.text), etree.HTMLParser())
        article_elements = parse_tree.xpath('//*[@id="article_body"]//p')
        post = []
        for a_part in article_elements:
            if type(a_part.text) is str:
                #each_article_text_list.append(a_part.text.strip())
                post.append(a_part.text.strip())
        each_article_text_list.append(post)
        #print(each_article_text_list)
        sleep_time = random.randint(3,10)
        print("sleep time: %s sec"%(sleep_time))
        time.sleep(sleep_time)
    #all_article_text = ''.join(each_article_text_list)
#all_article_text
each_article_text_list

https://money.udn.com/money/story/10511/6156480
sleep time: 3 sec


/opt/conda/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'money.udn.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


https://money.udn.com/money/story/10511/6155939
sleep time: 9 sec


/opt/conda/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'money.udn.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


https://money.udn.com/money/story/10511/6155038
sleep time: 3 sec


...

https://money.udn.com/money/story/10511/6137320
sleep time: 4 sec


/opt/conda/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'money.udn.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


https://money.udn.com/money/story/10511/5796972
sleep time: 8 sec


/opt/conda/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'money.udn.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


[['美國國防部助理部長卡琳（Mara Karlin） 周四在參議院聽證會上表示，若中國大陸選擇侵略侵犯台灣主權，烏克蘭對俄羅斯軍隊入侵的頑強抵抗，可作為台灣自衛的典範。',
  '',
  '負責戰略、計畫和軍力的助理國防部長卡琳說：「我認為我們現在在烏克蘭看到的情況，是對台灣非常有研究價值的案例，由此解釋為何台灣需要盡其所能建立非對稱能力，並讓民眾做好準備。若中國侵犯其主權，就可以盡可能地變得難以招惹。」',
  '',
  '美國長期以來一直敦促台灣購買具有成本效益的移動防衛系統，也就是所謂的「不對稱」防禦武器，以對抗中國大陸更強大的軍事力量。',
  '',
  '負責政治和軍事事務的助理國務卿露易絲對參議員表示，美國繼續敦促台灣採購「在烏克蘭發揮重大作用」的非對稱武器系統。',
  '',
  '露易絲也說，台灣還需要從烏克蘭的預備役部隊改革以及全民「須做好作戰準備」中汲取經驗。烏克蘭有領土防衛志願部隊和大約90萬後備役人員。',
  '',
  '她說，台灣不久前建立了全民防衛動員署，並在發展階段與美國國民兵合作。'],
 ['匈牙利國會今天選出總理奧班（Viktor Orban）的親密盟友諾瓦克（Katalin Novak）出任總統。她也成為匈牙利有史以來的首位女總統。',
  '諾瓦克表示，她的當選是女性的勝利。諾瓦克最近擔任的公職是家庭政策部長（minister for familypolicy）。',
  '此外，她也一直是政府政策代言人，包括稅收減免，以及旨在鼓勵年輕家庭生育更多子女的措施。',
  '在奧班的右翼「青年民主黨」（Fidesz）主導的國會中，諾瓦克以137票對51票當選僅具象徵意義的總統職位，擊敗反對黨挑戰者、經濟學家羅納（PeterRona）。',
  '諾瓦克在國會投票前的演說中表示：「我們深知話語的力量，但必要時我們也可以保持安靜並專注傾聽。一旦面對危險威脅，我們也會以超乎男性的勇氣，捍衛我們的家庭。」',
  '44歲的諾瓦克是匈牙利有史以來最年輕的國家元首。',
  '「這不僅是因為我想成為優秀的匈牙利總統，也是因為我是一名女性。」',
  '諾瓦克稍早在社群媒體上傳一張她和丈夫以及三名子女的合照，並寫下：「我的家人和我在一起，對我來說意義重大。」',
  '奧地利將於4月3日將舉行競爭激烈的國會選舉，自2010年掌權

In [15]:
# 存檔
with open(news_crawl_result_file, 'wb') as saveFile:
    for post in each_article_text_list:
        if len(post) > 0:
            for line in post:
                line = line.encode('utf-8')
                saveFile.write(line)
                saveFile.write('\n'.encode())

In [16]:
import jieba
import time
import multiprocessing
cores = multiprocessing.cpu_count() # Count the number of cores in a computer

In [17]:
!wget https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big -O /home/jovyan/work/jieba_data/dict.txt.big

--2022-03-11 11:56:20--  https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big
Resolving github.com (github.com)... 52.69.186.44
Connecting to github.com (github.com)|52.69.186.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/dict.txt.big [following]
--2022-03-11 11:56:20--  https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/dict.txt.big
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8583143 (8.2M) [text/plain]
Saving to: ‘/home/jovyan/work/jieba_data/dict.txt.big’

/home/jovyan/work/j 100%[===================>]   8.18M  2.97MB/s    in 2.8s    

2022-03-11 11:56:24 (2.97 MB/s) - ‘/home/jovyan/work/jieba_data/dict.txt.big’ saved [8583143/

In [18]:
!wget https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/stop_words.txt -O /home/jovyan/work/jieba_data/stop_words.txt

--2022-03-11 11:56:25--  https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/stop_words.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 222 [text/plain]
Saving to: ‘/home/jovyan/work/jieba_data/stop_words.txt’

/home/jovyan/work/j 100%[===================>]     222  --.-KB/s    in 0s      

2022-03-11 11:56:25 (11.5 MB/s) - ‘/home/jovyan/work/jieba_data/stop_words.txt’ saved [222/222]



In [19]:
jieba.set_dictionary('/home/jovyan/work/jieba_data/dict.txt.big')

In [20]:
jieba.load_userdict('/home/jovyan/work/jieba_data/mydict.txt')

Building prefix dict from /home/jovyan/work/jieba_data/dict.txt.big ...
Dumping model to file cache /tmp/jieba.ud3c40470d3518910bfd0dc86d865fad4.cache
Loading model cost 2.637 seconds.
Prefix dict has been built successfully.


In [21]:
# 開檔後一次讀一行
stop_words_list = []
with open(file='/home/jovyan/work/jieba_data/stop_words.txt',mode='r', encoding="UTF-8") as file:
    for line in file:
        line = line.strip()
        stop_words_list.append(line)
stop_words_list

['the',
 'of',
 'is',
 'and',
 'to',
 'in',
 'that',
 'we',
 'for',
 'an',
 'are',
 'by',
 'be',
 'as',
 'on',
 'with',
 'can',
 'if',
 'from',
 'which',
 'you',
 'it',
 'this',
 'then',
 'at',
 'have',
 'all',
 'not',
 'one',
 'has',
 'or',
 'that',
 '的',
 '了',
 '和',
 '是',
 '就',
 '都',
 '而',
 '及',
 '與',
 '著',
 '或',
 '一個',
 '沒有',
 '我們',
 '你們',
 '妳們',
 '他們',
 '她們',
 '是否']

### 分詞

In [27]:
#print(each_article_text_list)
seg_result = []
for line in each_article_text_list:
    line = ' '.join(line)
    seg_result.append(' '.join(jieba.lcut(line)))
#seg_result

In [28]:
seg_result = []
for post in each_article_text_list:
    for line in post:
        seg_result.append(' '.join(jieba.lcut(line)))
#seg_result

In [29]:
seg_stop_result = []
for line in seg_result:
    #print("[%s]"%(line))
    new_line = []
    for word in line.split():
        if word not in stop_words_list:
            #print(word)
            new_line.append(word)
    #print(new_line)
    seg_stop_result.append(' '.join(new_line))

In [30]:
'（' not in stop_words_list

True

In [31]:
'妳' not in stop_words_list

True

In [32]:
len(seg_stop_result)

177

In [33]:
seg_stop_result

['美國國防部 助理 部長 卡琳 （ Mara Karlin ） 周四 在 參議院 聽證會 上 表示 ， 若 中國 大陸 選擇 侵略 侵犯 台灣 主權 ， 烏克蘭 對 俄羅斯 軍隊 入侵 頑強抵抗 ， 可作 為 台灣 自衛 典範 。',
 '',
 '負責 戰略 、 計畫 軍力 助理 國防部長 卡琳 說 ： 「 我 認為 現在 在 烏克蘭 看到 情況 ， 對 台灣 非常 有 研究 價值 案例 ， 由此 解釋 為何 台灣 需要 盡其所能 建立 非對稱 能力 ， 並讓 民眾 做好 準備 。 若 中國 侵犯 其 主權 ， 可以 盡 可能 地 變得 難以 招惹 。 」',
 '',
 '美國 長期以來 一直 敦促 台灣 購買 具有 成本 效益 移動 防衛 系統 ， 也 就是 所謂 「 不 對稱 」 防禦 武器 ， 以 對抗 中國 大陸 更 強大 軍事力量 。',
 '',
 '負責 政治 軍事 事務 助理 國務卿 露易絲 對 參議員 表示 ， 美國 繼續 敦促 台灣 採購 「 在 烏克蘭 發揮 重大 作用 」 非對稱 武器系統 。',
 '',
 '露易絲 也 說 ， 台灣還 需要 從 烏克蘭 預備役 部隊 改革 以及 全民 「 須 做好 作戰 準備 」 中 汲取 經驗 。 烏克蘭 有 領土 防衛 志願 部隊 大約 90 萬 後備 役 人員 。',
 '',
 '她 說 ， 台灣 不久前 建立 全民 防衛 動員 署 ， 並在 發展 階段 美國 國民兵 合作 。',
 '匈牙利 國會 今天 選出 總理 奧班 （ Viktor Orban ） 親密 盟友 諾瓦克 （ Katalin Novak ） 出任 總統 。 她 也 成為 匈牙利 有史以來 首位 女 總統 。',
 '諾瓦克 表示 ， 她 當選 女性 勝利 。 諾瓦克 最近 擔任 公職 家庭 政策 部長 （ minister familypolicy ） 。',
 '此外 ， 她 也 一直 政府 政策 代言人 ， 包括 稅收 減免 ， 以及 旨在 鼓勵 年輕 家庭 生育 更 多 子女 措施 。',
 '在 奧班 右翼 「 青年 民主黨 」 （ Fidesz ） 主導 國會 中 ， 諾瓦克 以 137 票 對 51 票 當選 僅具 象徵意義 總統 職位 ， 擊敗 反對黨 挑戰者 、 經濟學家 羅納 （ PeterRona ）

In [34]:
#製作 sentence 的時候的樣子
for line in seg_stop_result:
    print(line.split())

['美國國防部', '助理', '部長', '卡琳', '（', 'Mara', 'Karlin', '）', '周四', '在', '參議院', '聽證會', '上', '表示', '，', '若', '中國', '大陸', '選擇', '侵略', '侵犯', '台灣', '主權', '，', '烏克蘭', '對', '俄羅斯', '軍隊', '入侵', '頑強抵抗', '，', '可作', '為', '台灣', '自衛', '典範', '。']
[]
['負責', '戰略', '、', '計畫', '軍力', '助理', '國防部長', '卡琳', '說', '：', '「', '我', '認為', '現在', '在', '烏克蘭', '看到', '情況', '，', '對', '台灣', '非常', '有', '研究', '價值', '案例', '，', '由此', '解釋', '為何', '台灣', '需要', '盡其所能', '建立', '非對稱', '能力', '，', '並讓', '民眾', '做好', '準備', '。', '若', '中國', '侵犯', '其', '主權', '，', '可以', '盡', '可能', '地', '變得', '難以', '招惹', '。', '」']
[]
['美國', '長期以來', '一直', '敦促', '台灣', '購買', '具有', '成本', '效益', '移動', '防衛', '系統', '，', '也', '就是', '所謂', '「', '不', '對稱', '」', '防禦', '武器', '，', '以', '對抗', '中國', '大陸', '更', '強大', '軍事力量', '。']
[]
['負責', '政治', '軍事', '事務', '助理', '國務卿', '露易絲', '對', '參議員', '表示', '，', '美國', '繼續', '敦促', '台灣', '採購', '「', '在', '烏克蘭', '發揮', '重大', '作用', '」', '非對稱', '武器系統', '。']
[]
['露易絲', '也', '說', '，', '台灣還', '需要', '從', '烏克蘭', '預備役', '部隊', '改革', '以及', '全民', '「', '須', '做

### word2vec 建模

In [35]:
from gensim.models import word2vec

In [36]:
class MySentence:
    def __init__(self, seg_result_list):
        self.seg_result_list = seg_result_list
        
    def __iter__(self):
        for line in self.seg_result_list:
            yield line.split()

In [37]:
sentence = MySentence(seg_stop_result)

In [38]:
start_time = time.time()
model = word2vec.Word2Vec( sentence , vector_size=300, sg=1, window=5, min_count=1, workers=cores-1)
print("--- spend %s seconds ---" % (time.time() - start_time))

--- spend 0.1929938793182373 seconds ---


### 視覺化

In [39]:
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np 

In [40]:
len(model.wv.vectors)

1956

In [41]:
def reduce_dimensions(model):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    # extract the words & their vectors, as numpy arrays
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  # fixed-width numpy strings

    # reduce using t-SNE
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


x_vals, y_vals, labels = reduce_dimensions(model)

/opt/conda/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


In [42]:
def plot_with_plotly(x_vals, y_vals, labels, plot_in_notebook=True):
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go

    trace = go.Scatter(x=x_vals, y=y_vals, mode='text', text=labels)
    data = [trace]

    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')


def plot_with_matplotlib(x_vals, y_vals, labels):
    import matplotlib.pyplot as plt
    import random

    random.seed(0)

    plt.figure(figsize=(12, 12))
    plt.scatter(x_vals, y_vals)

    #
    # Label randomly subsampled 25 data points
    #
    indices = list(range(len(labels)))
    selected_indices = random.sample(indices, 25)
    for i in selected_indices:
        plt.annotate(labels[i], (x_vals[i], y_vals[i]))

try:
    get_ipython()
except Exception:
    plot_function = plot_with_matplotlib
else:
    plot_function = plot_with_plotly

plot_function(x_vals, y_vals, labels)

In [43]:
model.wv.similar_by_word('拜登')

[('，', 0.9990276098251343),
 ('。', 0.9988003969192505),
 ('（', 0.9987727403640747),
 ('美國', 0.9987112879753113),
 ('）', 0.9986447095870972),
 ('在', 0.99860680103302),
 ('、', 0.998579740524292),
 ('對', 0.9985114932060242),
 ('大陸', 0.9985107183456421),
 ('「', 0.9985097050666809)]

In [45]:
model.wv.similar_by_word('烏克蘭')

[('，', 0.9988060593605042),
 ('大陸', 0.9985511898994446),
 ('（', 0.9985309839248657),
 ('。', 0.9985255002975464),
 ('）', 0.998497486114502),
 ('、', 0.9984591007232666),
 ('對', 0.9984574913978577),
 ('美國', 0.998418390750885),
 ('在', 0.998383104801178),
 ('可能', 0.9982775449752808)]

## 潛在語意分析 

In [46]:
len(seg_stop_result)

177

In [47]:
processed_corpus = []
for post in seg_stop_result:
    post_corpus = []
    for text in post.split():
        post_corpus.append(text)
    processed_corpus.append(post_corpus)

In [48]:
len(processed_corpus)

177

In [49]:
#一個句子一個 list, 全部的句子再用一個 list 包起來 
processed_corpus

[['美國國防部',
  '助理',
  '部長',
  '卡琳',
  '（',
  'Mara',
  'Karlin',
  '）',
  '周四',
  '在',
  '參議院',
  '聽證會',
  '上',
  '表示',
  '，',
  '若',
  '中國',
  '大陸',
  '選擇',
  '侵略',
  '侵犯',
  '台灣',
  '主權',
  '，',
  '烏克蘭',
  '對',
  '俄羅斯',
  '軍隊',
  '入侵',
  '頑強抵抗',
  '，',
  '可作',
  '為',
  '台灣',
  '自衛',
  '典範',
  '。'],
 [],
 ['負責',
  '戰略',
  '、',
  '計畫',
  '軍力',
  '助理',
  '國防部長',
  '卡琳',
  '說',
  '：',
  '「',
  '我',
  '認為',
  '現在',
  '在',
  '烏克蘭',
  '看到',
  '情況',
  '，',
  '對',
  '台灣',
  '非常',
  '有',
  '研究',
  '價值',
  '案例',
  '，',
  '由此',
  '解釋',
  '為何',
  '台灣',
  '需要',
  '盡其所能',
  '建立',
  '非對稱',
  '能力',
  '，',
  '並讓',
  '民眾',
  '做好',
  '準備',
  '。',
  '若',
  '中國',
  '侵犯',
  '其',
  '主權',
  '，',
  '可以',
  '盡',
  '可能',
  '地',
  '變得',
  '難以',
  '招惹',
  '。',
  '」'],
 [],
 ['美國',
  '長期以來',
  '一直',
  '敦促',
  '台灣',
  '購買',
  '具有',
  '成本',
  '效益',
  '移動',
  '防衛',
  '系統',
  '，',
  '也',
  '就是',
  '所謂',
  '「',
  '不',
  '對稱',
  '」',
  '防禦',
  '武器',
  '，',
  '以',
  '對抗',
  '中國',
  '大陸',
  '更',
  '強大',
  '軍事力量',
  '。'],
 

In [50]:
from gensim import corpora
#建立辭典
dictionary = corpora.Dictionary(processed_corpus, prune_at=100)
print(dictionary)

Dictionary(1956 unique tokens: ['Karlin', 'Mara', '。', '上', '中國']...)


In [51]:
#pprint.pprint(dictionary.token2id)

### 每個()中的第一個數字對應於字典中token 的ID，第二個數字對應於此token的計數。

In [52]:
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus ]
#pprint.pprint(bow_corpus)

In [53]:
new_text = "拜登 阿富汗"
#new_text = "台灣"

In [54]:
new_vec = dictionary.doc2bow(new_text.split())
print(new_vec)

[(384, 1)]


## 建立 tf-idf 模型

In [55]:
from gensim import models

In [56]:
tfidf = models.TfidfModel(bow_corpus)

In [57]:
corpus_tfidf = tfidf[bow_corpus]

In [58]:
for i in corpus_tfidf:
    pprint.pprint(i)

[(0, 0.2277499493297361),
 (1, 0.2277499493297361),
 (2, 0.011593740737115426),
 (3, 0.11163175257129818),
 (4, 0.08791597306858),
 (5, 0.19725155894177507),
 (6, 0.17941114423246327),
 (7, 0.19725155894177507),
 (8, 0.12224285330407972),
 (9, 0.14213014295925924),
 (10, 0.2277499493297361),
 (11, 0.17941114423246327),
 (12, 0.19725155894177507),
 (13, 0.16675316855381395),
 (14, 0.2277499493297361),
 (15, 0.18758267572397275),
 (16, 0.2277499493297361),
 (17, 0.04909095805507868),
 (18, 0.09174446291611864),
 (19, 0.06887000687690666),
 (20, 0.11841436345654109),
 (21, 0.09379133786198637),
 (22, 0.2277499493297361),
 (23, 0.2277499493297361),
 (24, 0.2277499493297361),
 (25, 0.19725155894177507),
 (26, 0.08611981034843708),
 (27, 0.2277499493297361),
 (28, 0.2277499493297361),
 (29, 0.19725155894177507),
 (30, 0.2277499493297361),
 (31, 0.04909095805507868),
 (32, 0.04909095805507868),
 (33, 0.03772557394348795)]
[]
[(2, 0.01808108178851013),
 (4, 0.06855491836558665),
 (5, 0.1538123

In [59]:
tfidf[new_vec]

[(384, 1.0)]

## 建立LSI 模型

In [60]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary)

In [61]:
#LSI(潛在語意索引)
corpus_lsi = lsi[corpus_tfidf]

In [62]:
#目前的主題
lsi.print_topics()

[(0,
  '0.203*"關稅" + 0.177*"，" + 0.168*"美國" + 0.146*"政府" + 0.143*"可能" + 0.143*"在" + 0.141*"大陸" + 0.126*"、" + 0.124*"新" + 0.122*"拜登"'),
 (1,
  '-0.225*"法案" + 0.198*"關稅" + -0.160*"台灣" + 0.141*"可能" + -0.134*"（" + -0.134*"）" + -0.130*"通過" + 0.126*"大陸" + 0.125*"政府" + 0.121*"補貼"'),
 (2,
  '0.360*"關稅" + -0.241*"補貼" + -0.206*"產業" + -0.162*"加強" + 0.134*"商品" + -0.132*"知情" + -0.131*"技術" + -0.128*"新" + 0.121*"檢討" + 0.118*"政府"'),
 (3,
  '0.324*"台灣" + 0.229*"加拿大" + -0.212*"法案" + 0.193*"CPTPP" + 0.186*"加入" + -0.116*"通過" + 0.106*"經濟" + 0.102*"夥伴" + -0.102*"同性戀" + 0.100*"台加"'),
 (4,
  '0.175*"川普" + 0.175*"紐奧良" + 0.171*"委員會" + 0.161*"飛機" + 0.150*"故障" + -0.150*"法案" + -0.143*"台灣" + 0.131*"後" + 0.131*"5" + 0.123*"共和黨"'),
 (5,
  '0.221*"烏克蘭" + 0.150*"普里" + 0.150*"馬" + 0.150*"琴科" + 0.145*"俄羅斯" + 0.122*"也" + 0.119*"她" + -0.114*"產業" + 0.110*"入侵" + -0.098*"台灣"'),
 (6,
  '0.310*"加強" + 0.180*"技術" + 0.171*"企業" + -0.160*"可能" + 0.147*"敏感" + 0.147*"出口" + 0.147*"管制" + 0.138*"在" + 0.133*"歐亞" + -0.132*"新"'),
 (7,
  '0.1

In [63]:
#新文章
new_doc = '拜登近期因對撤離過程的處理而受到猛烈批評。最近幾天，撤離過程凸顯出喀布爾機場的混亂場面，人群拼命想離開這個國家。但拜登辯護，稱結束美國在阿富汗20年的參與不可避免的會出現問題。'
#new_vec_bow = dictionary.doc2bow()

In [64]:
#分詞
new_seg_result = jieba.lcut(new_doc)
len(new_seg_result)

50

In [65]:
new_doc_bow = dictionary.doc2bow(new_seg_result)

In [66]:
new_doc_lsi = lsi[new_doc_bow]

In [67]:
pprint.pprint(new_doc_lsi)

[(0, 1.7405523257109035),
 (1, -0.3746009094704735),
 (2, 0.3484429597287864),
 (3, -0.007891286423163536),
 (4, 0.35089690592032946),
 (5, 0.3838471015169459),
 (6, 0.27157106114014706),
 (7, -0.1103044476689388),
 (8, 0.40998155374306944),
 (9, -0.010117167645990957),
 (10, 0.0743946088420237),
 (11, 0.009801647138856947),
 (12, -0.24414581183972528),
 (13, 0.23302241105477348),
 (14, -0.2015793522211038),
 (15, -0.34196682328371364),
 (16, 0.06819018642913383),
 (17, -0.10323012977210924),
 (18, -0.03421731454019196),
 (19, 0.16220682362292388),
 (20, -0.042354307215548295),
 (21, 0.12585168872753766),
 (22, 0.08023251106199313),
 (23, -0.28516442481579574),
 (24, 0.12071764063253579),
 (25, 0.04088745268081422),
 (26, -0.0001898483189370949),
 (27, 0.05297650492823738),
 (28, 0.19371964774966335),
 (29, -0.09808845119518905),
 (30, -0.0021168876675110625),
 (31, 0.2907340859046639),
 (32, 0.1909114060315299),
 (33, -0.12362237646238969),
 (34, -0.05568041110580223),
 (35, 0.0377177

### 建立索引

In [68]:
from gensim import similarities

In [69]:
index = similarities.MatrixSimilarity(corpus_lsi)

In [70]:
index

In [71]:
#新文章相似度
sims = index[new_doc_lsi]

In [72]:
sims

array([0.07785547, 0.        , 0.08304902, 0.        , 0.06845336,
       0.        , 0.08464617, 0.        , 0.03747883, 0.        ,
       0.09557976, 0.02916052, 0.17637366, 0.07346018, 0.06097743,
       0.16684009, 0.0249301 , 0.1320739 , 0.07896647, 0.08663347,
       0.09997822, 0.02820962, 0.07938156, 0.04187619, 0.03542632,
       0.1223687 , 0.03425236, 0.03987881, 0.16306025, 0.08372621,
       0.21100695, 0.20752649, 0.14334622, 0.05929617, 0.07220345,
       0.08577017, 0.07773938, 0.05838807, 0.06752819, 0.07842426,
       0.0650351 , 0.17599705, 0.04185963, 0.0634933 , 0.04009075,
       0.1169562 , 0.19584684, 0.24225798, 0.        , 0.3550095 ,
       0.        , 0.17072105, 0.        , 0.0725252 , 0.10933839,
       0.18598221, 0.06831412, 0.06564777, 0.11053649, 0.17521131,
       0.08742657, 0.24225171, 0.08395639, 0.0515926 , 0.3473275 ,
       0.26086068, 0.14556599, 0.14774325, 0.18020351, 0.08721922,
       0.0652983 , 0.13150501, 0.        , 0.1713934 , 0.     

In [73]:
print(list(enumerate(sims))) # print (document_number, document_similarity) 2-tuples

[(0, 0.07785547), (1, 0.0), (2, 0.08304902), (3, 0.0), (4, 0.06845336), (5, 0.0), (6, 0.08464617), (7, 0.0), (8, 0.037478827), (9, 0.0), (10, 0.09557976), (11, 0.02916052), (12, 0.17637366), (13, 0.07346018), (14, 0.060977425), (15, 0.16684009), (16, 0.024930103), (17, 0.1320739), (18, 0.07896647), (19, 0.08663347), (20, 0.099978216), (21, 0.028209617), (22, 0.07938156), (23, 0.041876193), (24, 0.035426322), (25, 0.1223687), (26, 0.03425236), (27, 0.039878808), (28, 0.16306025), (29, 0.08372621), (30, 0.21100695), (31, 0.20752649), (32, 0.14334622), (33, 0.059296172), (34, 0.07220345), (35, 0.08577017), (36, 0.07773938), (37, 0.05838807), (38, 0.06752819), (39, 0.07842426), (40, 0.0650351), (41, 0.17599705), (42, 0.041859634), (43, 0.0634933), (44, 0.040090747), (45, 0.116956204), (46, 0.19584684), (47, 0.24225798), (48, 0.0), (49, 0.3550095), (50, 0.0), (51, 0.17072105), (52, 0.0), (53, 0.0725252), (54, 0.10933839), (55, 0.18598221), (56, 0.06831412), (57, 0.065647766), (58, 0.1105364

In [74]:
sims = sorted(enumerate(sims), key=lambda i: -i[1])

In [75]:
for doc_pos, doc_score in sims[:5]:
    print(doc_score, seg_stop_result[doc_pos])

0.3550095 美國 政治 新聞 網站 Politico 9 日 報導 ， 消息人士 指出 ， 川普 5 日 晚間 結束 共和黨 全國 委員會 餐會 後 搭 上 專機 ， 但 在 飛行 約 20 至 30 分鐘 後 ， 其中 引擎 突然 發生 故障 ， 機長 決定 掉頭 飛往 紐奧良 機場 。
0.3473275 這項 法案 從州眾 議院 到州 參議院 立法 過程 中 ， 受到 LGBTQ （ 同性戀 、 雙性戀 、 跨 性別 酷兒 ） 團體 抗議 。 美國 總統 拜登 （ Joe Biden ） 白宮 官方 帳號 2 月 8 日均 在 推特上 發聲 譴責 這項 在 佛州 立法 。
0.28907475 雖然 新 301 調查 可能 會 引發 拜登 政府 修改 川普 執政 時期 關稅 政策 ， 但 貿易 專家 表示 ， 鑒 於 大陸 未能 達到 第一階段 貿易 協議 中 採購 目標 ， 拜登 政府 目前 可能 不會 考慮 對 大幅 削減 任何 關稅 。
0.28382117 近來 美企已 施壓 拜登 政府 調整 對陸 產品 加徵 關稅 計畫 ， 儘管 新 301 調查 可能 促使 拜登 政府 修改 關稅 政策 ， 但 貿易 專家 表示 ， 由於 中方 未能 達到 第一階段 貿易 協議 採購 目標 ， 拜登 政府 目前 可能 不會 考慮 大幅 削減 任何 關稅 。
0.2758047 川普 第一批 對陸 懲罰 關稅 ， 在 2018 年 3 到 4 月間 對 340 億美元 大陸 產品 加徵 關稅 ， 評估 過程 需在 60 天內 完成 ， 因此 美國 貿易 代表 署須 在 7 月 6 日前 ， 提出 第一批 懲罰 關稅 檢討 報告 。


In [76]:
print(sims[:5])

[(49, 0.3550095), (64, 0.3473275), (152, 0.28907475), (135, 0.28382117), (96, 0.2758047)]
